# Indexing examples for wdl file multi-dimensional Arrays
```bash
docker run -p 8888:8888 -v `pwd`:/home/tmpusr/run_dir -it aka1spanky/ncsa:devwdlV01 jupyter notebook --port=8888 --ip=0.0.0.0 --allow-root
```

In [2]:
import os
import sys
import numpy as np
import pandas as pd

In [3]:
# running jupyter inside the container - this is the externally mounted directory
os.getcwd()

'/home/tmpusr/run_dir'

In [4]:
#           location of the cromwell files
os.listdir('/usr/locl/etc')

['womtool-36.jar', 'cromwell-36.jar', '.DS_Store']

In [5]:
#           before and after cell >> cromwell-* will show up after running the next cells
# os.listdir('cromwell-executions/wf_hello')  # for subsequent runs
os.listdir()

['T2IdxArray.wdl',
 'HelloIdxArray.wdl',
 'Docker_Container_Run.ipynb',
 'AAA_json.json',
 '.ipynb_checkpoints',
 'cromwell-executions',
 'cromwell-workflow-logs']

## Iterative Develop-Test: edit wdl file in next cell and re-run it and the next one.

In [39]:
with open('IdxArray.wdl', 'r') as fh:
    lines = fh.readlines()
for line in lines:
    print(line)

###########################

#       IdxArray.wdl

# Usage: 

# java -jar full/path/to/cromwell run IdxArray.wdl

###########################

task rep_or_t {

    Int dakine

    Int dakineidx

    command {

        echo "N = ${dakine}"}}



workflow wf_hello {

    Array[Array[Array[String]]] ImpString = [ [ ["0:0:0", "0:0:1"], ["0:1:0","0:1:1"] ],  [ ["1:0:0", "1:0:1"], ["1:1:0","1:1:1"] ] ]

    scatter (idx in outer_arr_idx) {

        call rep_or_t as do_da_report {

             input: 

                 dakine = idx,

                 dakineidx = length(ImpString[idx]

        }

    }

}



```bash
    Array[Array[Array[String]]] ImpString = [
    [
    ["0:0:0","0:0:1"],["0:1:0","0:1:1"]
    ], [
    ["1:0:0","1:0:1"],["1:1:0","1:1:1"]
    ]
    ]
```
## Demonstrates that scatter blocks do not execute in any particular order:
* perhaps with output dependancies ?
* multiple workflows w dependancies ?


In [100]:
%%writefile IdxArray.wdl
###########################
#       IdxArray.wdl
# Usage: 
# java -jar full/path/to/cromwell run IdxArray.wdl
###########################
task rep_or_t {
    Int dakine
    Int dakineidx
    command {
        echo "N[${dakine}] = ${dakineidx}"
    }
}

task p_or_t {
    Int dakine
    Int dakineidx
    command {
        echo "N[${dakine}] = ${dakineidx}"
    }
}


workflow wf_hello {
    Array[Array[Array[String]]] ImpString = [[["0:0:0","0:0:1"],["0:0:0","0:0:1"],["0:1:0","0:1:1"]],
                                             [["1:1:0","1:1:1"]],
                                             [["1:0:0","1:0:1"],["1:1:0","1:1:1"]]]
    Array[Int] AAA_size = [3,1,2] 
    Int arr_size = length(ImpString)
    Int leaf_size = length(ImpString[0][0])
    Int number_of_elements = AAA_size[0] * AAA_size[1] * AAA_size[2]
    Int number_of_lanes = number_of_elements / leaf_size
    Array[Int] arr_idx = range( number_of_elements)

    scatter (idx in arr_idx) {
        
        Int ix3 = idx % leaf_size
        Int ix2 = (idx / number_of_lanes) % 3
        Int lane_size = AAA_size[ix2]
        
        call rep_or_t as do_da_report {

             input: 
                dakine = lane_size,
                dakineidx = ix3
        }
    }
}


Overwriting IdxArray.wdl


In [101]:
#           construct the call string and start a new separate process
hak_str = 'java -jar /usr/locl/etc/cromwell-36.jar run IdxArray.wdl'
os.system(hak_str)

0

In [8]:
%%writefile T2IdxArray.wdl
###########################
#              AAA_T2IdxArray.wdl
# Usage:
# java -jar full/path/to/cromwell run AAA_T2IdxArray.wdl -i AAA_json.json
# wf_hello.ImpString = [[["1"]]] 
###########################

task hello {
  String addressee
  Int One
  Int Two
  Int Three
  Int sIx
  command {
    echo "Input String =  ${addressee} With Iterator Index: ${sIx}  [${One}][${Two}][${Three}]"
  }
  output {
    String message = read_string(stdout())
  }
}

workflow wf_hello {

  Array[Array[Array[String]]] ImpString
  Int arr1_size = length(ImpString)
  Int arr2_size = length(ImpString[0])
  Int arr3_size = length(ImpString[0][0])

  Array[Int] Index = range(arr1_size * arr2_size * arr3_size)

  scatter (idx in Index) {

    Int reads_idx = idx % arr3_size

    Int lanes_idx = (idx / arr2_size) % arr2_size

    Int samples_idx = (idx / (arr3_size * arr2_size)) % arr1_size

    call hello as howdy {
      input:
        addressee = ImpString[samples_idx][lanes_idx][reads_idx],
        One = samples_idx,
        Two = lanes_idx,
        Three = reads_idx,
        sIx = idx
    }
  }

  output {
     howdy.message
  }
}


Overwriting T2IdxArray.wdl


In [9]:
%%writefile AAA_json.json
{
    "wf_hello.ImpString": [ 
                          [ ["0:0:0", "0:0:1"], 
                            ["0:1:0","0:1:1"] ],  
                          [ ["1:0:0", "1:0:1"], 
                            ["1:1:0","1:1:1"] ], 
                          [ ["2:0:0", "2:0:1"], 
                            ["2:1:0","2:1:1"] ] ]
}


Overwriting AAA_json.json


In [10]:
#           construct the call string and start a new separate process
hak_str = 'java -jar /usr/locl/etc/cromwell-36.jar run T2IdxArray.wdl -i AAA_json.json'
os.system(hak_str)

0

In [11]:
#           before and after cell >> cromwell-* will show up after running the next cell
#           Iff and Only If you run the writefile below 
os.listdir('cromwell-executions/wf_hello')

['c773d129-8092-4095-876a-10e7c70b555f',
 'cae6045a-41cb-4d28-8934-b3ba880bffe4',
 'fdcb2cdd-f3bf-432b-be8f-669f8b687e91',
 '420bbb21-ea59-4c5b-8aff-affbcb0f9a80']